In [ ]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd
from tqdm import tqdm
import os
import os
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Change the working directory to your desired path
# Example: change to a directory within your Google Drive
os.chdir('/content/drive/MyDrive') # Replace with your folder path



In [ ]:
# path = '../DyAD/dyad_vae_save'
# path_list = os.listdir('../DyAD/dyad_vae_save')
# data = '2022-02-23-22-43-46_fold1'
# test_res_csv_fold0_path = os.path.join(os.path.join(path,data),'result/test_segment_scores.csv')
# train_res_csv_fold0_path = os.path.join(os.path.join(path,data),'result/train_segment_scores.csv')

# You should change the following path.

In [ ]:
test_res_csv_fold0 = pd.read_csv('/content/test_segment_scores.csv')
"""
test_res_csv_fold1 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-46_fold1/result/test_segment_scores.csv')
test_res_csv_fold2 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-51_fold2/result/test_segment_scores.csv')
test_res_csv_fold3 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-41-48_fold3/result/test_segment_scores.csv')
test_res_csv_fold4 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-48_fold4/result/test_segment_scores.csv')
"""

train_res_csv_fold0 = pd.read_csv('/content/train_segment_scores.csv')

"""
train_res_csv_fold1 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-46_fold1/result/train_segment_scores.csv')
train_res_csv_fold2 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-51_fold2/result/train_segment_scores.csv')
train_res_csv_fold3 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-41-48_fold3/result/train_segment_scores.csv')
train_res_csv_fold4 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-48_fold4/result/train_segment_scores.csv')
"""

"\ntrain_res_csv_fold1 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-46_fold1/result/train_segment_scores.csv')\ntrain_res_csv_fold2 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-51_fold2/result/train_segment_scores.csv')\ntrain_res_csv_fold3 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-41-48_fold3/result/train_segment_scores.csv')\ntrain_res_csv_fold4 = pd.read_csv('../DyAD/dyad_vae_save/2022-02-23-22-43-48_fold4/result/train_segment_scores.csv')\n"

In [ ]:
# replace the following two car number lists
ind_ood_car_dict = np.load('./five_fold_utils/ind_odd_dict2.npz.npy', allow_pickle=True).item()
ind_car_num_list = ind_ood_car_dict['ind_sorted']
ood_car_num_list = ind_ood_car_dict['ood_sorted']
all_car_num_list = set(ind_car_num_list + ood_car_num_list)

print(len(ind_car_num_list))
print(len(ood_car_num_list))

33
16


In [ ]:
#차량 단위 평균을 기반으로 ROC-AUC, threshold, F1, recall, false alarm rate(FA)

AUC_fivefold_list = []
# for i in range(5):
for i in range(1):
    fold_num = i

    validation_car_number = ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)] + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):] + ood_car_num_list[int(fold_num * len(ood_car_num_list) / 5):int((fold_num + 1) * len(ood_car_num_list) / 5)]
    # test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]
    test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]

    data = np.vstack((np.array(eval('train_res_csv_fold%d' % i)), np.array(eval('test_res_csv_fold%d' % i))))[:, 1:]
#     print(data[0:5])

    ind_car_score = []
    ood_car_score = []

    for each_car_num in test_car_number:
        this_car_score = np.mean(data[np.where(data[:, 0]==each_car_num)][:, 2])
        if each_car_num in ind_car_num_list:
            ind_car_score.append(this_car_score)
        else:
            ood_car_score.append(this_car_score)
    #     print(each_car_num, len(this_car_score))

    label = [1] * len(ood_car_score) + [0] * len(ind_car_score)
    score = np.hstack((ood_car_score, ind_car_score))

    fpr, tpr, thresholds = metrics.roc_curve(label, score, pos_label=1)
#    plt.plot(fpr, tpr)
#    plt.show()
    AUC = auc(fpr, tpr)
    print(AUC)

    # 최적 threshold 계산 (Youden's J statistic)
    J_scores = tpr - fpr  # Youden's J = TPR - FPR
    best_threshold_idx = np.argmax(J_scores)  # J 값이 최대인 인덱스 찾기
    best_threshold = thresholds[best_threshold_idx]

    print(f"Best threshold for fold {fold_num}: {best_threshold}")


    # 최적 threshold에서의 예측값
    predictions = (score >= best_threshold).astype(int)  # threshold 이상이면 1 (이상), 아니면 0 (정상)

    # confusion matrix 계산
    cm = confusion_matrix(label, predictions)
    tn, fp, fn, tp = cm.ravel()

    # Recall, FA, F1 Score 계산
    recall = tp / (tp + fn)
    fa = fp / (fp + tn)
    precision = tp / (tp + fp)
    f1_score = 2 * (precision * recall) / (precision + recall)

    # 결과 출력
    print(f"Recall: {recall:.4f}")
    print(f"False Alarm Rate (FA): {fa:.4f}")
    print(f"F1 Score: {f1_score:.4f}")


    AUC_fivefold_list.append(AUC)

print('AUC mean ', np.mean(AUC_fivefold_list))


0.9102564102564102
Best threshold for fold 0: 0.006529311182278576
Recall: 0.8462
False Alarm Rate (FA): 0.0000
F1 Score: 0.9167
AUC mean  0.9102564102564102


In [ ]:
# Robust Scoring Evaluation with AUC, Threshold, F1, Recall, FA

import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import auc, confusion_matrix

AUC_fivefold_list = []

# Define p-percentile for robust scoring
P_PERCENTILE = 90  # Top 10% percentile

# for i in range(5):
for i in range(1):
    fold_num = i

    validation_car_number = ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)] + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):] + ood_car_num_list[int(fold_num * len(ood_car_num_list) / 5):int((fold_num + 1) * len(ood_car_num_list) / 5)]
    test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]

    data = np.vstack((np.array(eval('train_res_csv_fold%d' % i)), np.array(eval('test_res_csv_fold%d' % i))))[:, 1:]

    ind_car_score = []
    ood_car_score = []

    for each_car_num in test_car_number:
        car_errors = data[np.where(data[:, 0] == each_car_num)][:, 2]
        # Robust scoring: top p percentile average
        p_threshold = np.percentile(car_errors, P_PERCENTILE)
        robust_score = np.mean(car_errors[car_errors >= p_threshold])
        if each_car_num in ind_car_num_list:
            ind_car_score.append(robust_score)
        else:
            ood_car_score.append(robust_score)

    label = [1] * len(ood_car_score) + [0] * len(ind_car_score)
    score = np.hstack((ood_car_score, ind_car_score))

    fpr, tpr, thresholds = metrics.roc_curve(label, score, pos_label=1)
    AUC = auc(fpr, tpr)
    print(f"Robust Scoring AUC (Fold {fold_num}): {AUC:.4f}")

    # Optimal threshold calculation (Youden's J statistic)
    J_scores = tpr - fpr
    best_threshold_idx = np.argmax(J_scores)
    best_threshold = thresholds[best_threshold_idx]

    print(f"Best threshold for fold {fold_num}: {best_threshold}")

    # Predictions using the robust threshold
    predictions = (score >= best_threshold).astype(int)

    # Confusion matrix calculation
    cm = confusion_matrix(label, predictions)
    tn, fp, fn, tp = cm.ravel()

    # Recall, FA, F1 Score calculation
    recall = tp / (tp + fn)
    fa = fp / (fp + tn)
    precision = tp / (tp + fp)
    f1_score = 2 * (precision * recall) / (precision + recall)

    # Output the results
    print(f"Recall: {recall:.4f}")
    print(f"False Alarm Rate (FA): {fa:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    AUC_fivefold_list.append(AUC)

print('AUC mean ', np.mean(AUC_fivefold_list))


Robust Scoring AUC (Fold 0): 0.8974
Best threshold for fold 0: 0.018541733038268546
Recall: 0.8462
False Alarm Rate (FA): 0.0000
F1 Score: 0.9167
AUC mean  0.8974358974358976
